# Divide data into test/training sets

In [ ]:
from pathlib import Path
import pickle

In [ ]:
import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

# Import Matched Trace Data
Dictionary Containing Origins and Destinations, Matches, Shortest Paths

In [ ]:
#export for impedance calibration
with (config['calibration_fp']/'ready_for_calibration.pkl').open('rb') as fh:
    match_results = pickle.load(fh)
with (config['cycleatl_fp']/'trips_3.pkl').open('rb') as fh:
    reduced_trip_set = pickle.load(fh)
filter_trips = set(reduced_trip_set['tripid'].tolist())

In [ ]:
match_results = {key:item for key, item in match_results.items() if key in filter_trips}
print(len(match_results))

In [ ]:
def aggregate_trips_to_links(links,matched_traces,tripids,name):
    if tripids is not None:
        #use list of tripids to subset dict of matched traces
        filtered_dict = {key:item for key,item in matched_traces.items() if key in set(tripids)}
    else:
        filtered_dict = matched_traces
    #make one large series
    list_of_links = [item['edges'] for key, item in filtered_dict.items()]
    list_of_links = list(itertools.chain(*list_of_links))
    series_of_links = pd.Series(list_of_links)
    links[name] = links['linkid'].map(series_of_links.value_counts())
    return links

In [ ]:
import pandas as pd
directed_links = pd.read_parquet(config['network_fp']/'directed_edges.parquet')

aggregated_undirected_matched = { key : [] for key in directed_links['linkid'].tolist()}
aggregated_directed_matched = { key : [] for key in list(zip(directed_links['linkid'],directed_links['reverse_link']))}

In [ ]:
for tripid, item in match_results.items():
    matched_edges = item['matched_edges']
    matched_edges = [tuple(x) for x in matched_edges.values]
    for edge in matched_edges:
        aggregated_directed_matched[edge].append(tripid)
    
    matched_edges = item['matched_edges']
    matched_edges = matched_edges['linkid'].tolist()
    for edge in matched_edges:
        aggregated_undirected_matched[edge].append(tripid)
    


In [ ]:
#filter to only links where trips were observed
aggregated_directed_matched = {key:item for key, item in aggregated_directed_matched.items() if len(item) > 0}
aggregated_undirected_matched = {key:item for key, item in aggregated_undirected_matched.items() if len(item) > 0}

In [ ]:
aggregated_undirected_matched

In [ ]:
import geopandas as gpd
links = gpd.read_file(config['network_fp']/'final_network.gpkg',layer='edges')


In [ ]:
#add basic count
#links.set_index('linkid',inplace=True)
links['trips'] = links['linkid'].apply(lambda x: len(aggregated_undirected_matched.get(x,[])))
links['trips'].describe()

In [ ]:
only_trips = links[links['trips']>0]
only_trips.to_file(config['calibration_fp']/'scratch.gpkg',layer='link_counts')

In [ ]:
only_trips.columns

In [ ]:
only_trips.groupby('highway')[['trips','length_ft']].sum().sort_values('length_ft')

## Split the data into training and testing set
Think more about how to split the data

In [ ]:
# import random
# def get_random_keys(dictionary,num_keys):
#     random_keys = set()
#     while len(random_keys) < num_keys:
#         random_key =  random.choice(list(dictionary.keys()))
#         if isinstance(dictionary.get(random_key),str):
#             continue
#         if random_key not in random_keys:
#             random_keys.add(random_key)
#     return list(random_keys)

In [ ]:
# total_trips = 1000
# random_keys = get_random_keys(match_results,total_trips)

# middle_point = int(total_trips/2)

# train_set = random_keys[0:middle_point]
# test_set = random_keys[middle_point:]

In [ ]:
# #subset accordingly
# train_set = {key:item for key, item in match_results.items() if key in train_set}
# test_set = {key:item for key, item in match_results.items() if key in test_set}

In [ ]:
with (config['calibration_fp']/'full_set.pkl').open('wb') as fh:
    pickle.dump(match_results,fh)
# with (config['calibration_fp']/'test_set.pkl').open('wb') as fh:
#     pickle.dump(train_set,fh)
# with (config['calibration_fp']/'train_set.pkl').open('wb') as fh:
#     pickle.dump(test_set,fh)